In [1]:
from utils import create_dataloaders
from evaluate import reconstruction_loss, evaluate_model
from AutoDecoder import AutoDecoder
from trainer import BasicTrainer

1.3 Auto Decoder

In [2]:
train_ds, train_dl, test_ds, test_dl = create_dataloaders(data_path='', batch_size=64)


In [3]:
import torch
import torch.nn as nn


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

latent_dim = 256

model = AutoDecoder(latent_dim)
trainer = BasicTrainer(model=model, dl=train_dl, latent_dim=model.latent_dim, device=device)
trainer.train(num_epochs=180)

# Evaluation on the test dataset
latents = torch.nn.Parameter(torch.randn(len(test_dl.dataset), model.latent_dim).to(device))
opt = torch.optim.Adam([latents], lr=3e-3)
test_loss = evaluate_model(model=model, test_dl=test_dl, opt=opt, latents=latents, epochs=1000, device=device)
print(f"AD has finished test evaluation with a test loss of {test_loss}.")

Using device: cuda
Epoch [1/180], Loss: 0.4975
Epoch [2/180], Loss: 0.4512
Epoch [3/180], Loss: 0.4065
Epoch [4/180], Loss: 0.3620
Epoch [5/180], Loss: 0.3284
Epoch [6/180], Loss: 0.3080
Epoch [7/180], Loss: 0.3010
Epoch [8/180], Loss: 0.2956
Epoch [9/180], Loss: 0.2921
Epoch [10/180], Loss: 0.2852
Epoch [11/180], Loss: 0.2783
Epoch [12/180], Loss: 0.2729
Epoch [13/180], Loss: 0.2712
Epoch [14/180], Loss: 0.2646
Epoch [15/180], Loss: 0.2578
Epoch [16/180], Loss: 0.2520
Epoch [17/180], Loss: 0.2470
Epoch [18/180], Loss: 0.2437
Epoch [19/180], Loss: 0.2383
Epoch [20/180], Loss: 0.2346
Epoch [21/180], Loss: 0.2316
Epoch [22/180], Loss: 0.2287
Epoch [23/180], Loss: 0.2258
Epoch [24/180], Loss: 0.2246
Epoch [25/180], Loss: 0.2220
Epoch [26/180], Loss: 0.2201
Epoch [27/180], Loss: 0.2187
Epoch [28/180], Loss: 0.2164
Epoch [29/180], Loss: 0.2144
Epoch [30/180], Loss: 0.2116
Epoch [31/180], Loss: 0.2095
Epoch [32/180], Loss: 0.2092
Epoch [33/180], Loss: 0.2071
Epoch [34/180], Loss: 0.2061
Epoc

In [4]:
# Randomly sample 5 indices from the test dataset
import random
import utils
random.seed(6)
sampled_indices = random.sample(range(len(latents)), 5)

# Extract the corresponding vectors (input data) and their labels
sampled_latents = [latents[i] for i in sampled_indices]  # Only selecting input data, not labels

# Convert to a single tensor (optional)
sampled_latents_tensor = torch.stack(sampled_latents)
random_latents_tensor = torch.randn_like(sampled_latents_tensor)

print("Sampled Vectors Shape:", sampled_latents_tensor.shape)  # Should be (5, *) depending on your data shape
print("Random Vectors Shape:", random_latents_tensor.shape)  # Should be (5, *) depending on your data shape

sampled_test_images = model(sampled_latents_tensor).view(-1, 1, 28, 28)
random_test_images = model(random_latents_tensor).view(-1, 1, 28, 28)

print("Sampled Images Shape:", sampled_test_images.shape)  # Should be (5, *) depending on your data shape
utils.save_images(sampled_test_images, "sampled_test_images.png")
utils.save_images(random_test_images, "random_test_images.png")
# import matplotlib.pyplot as plt
# plt.imshow(sampled_test_images.detach().cpu().numpy().reshape(28, 28), cmap='gray')
# plt.show()

Sampled Vectors Shape: torch.Size([5, 256])
Random Vectors Shape: torch.Size([5, 256])
Sampled Images Shape: torch.Size([5, 1, 28, 28])


In [1]:
from utils import plot_tsne
# def plot_tsne(dataset, latents, file_name, plot_title="t-SNE Plot"):
plot_tsne(test_ds, latents, "just a file", plot_title="t-SNE Plot")


NameError: name 'test_ds' is not defined

VAD Part

In [ ]:
from utils import create_dataloaders
from evaluate import reconstruction_loss, evaluate_model
from VariationalAutoDecoder import VariationalAutoDecoder
from trainer import VADTrainer

In [ ]:
train_ds, train_dl, test_ds, test_dl = create_dataloaders(data_path='', batch_size=32)


In [ ]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

latent_dim = 128

model = VariationalAutoDecoder(latent_dim, device=device)

trainer = VADTrainer(model=model, dl=train_dl, latent_dim=model.latent_dim, device=device)

trainer.train(num_epochs=555)


# Evaluation on the test dataset
# latents = torch.nn.Parameter(torch.randn(len(test_dl.dataset), model.latent_dim).to(device))
# opt = torch.optim.Adam([latents], lr=3e-3)
# test_loss = evaluate_model(model=model, test_dl=test_dl, opt=opt, latents=latents, epochs=1000, device=device)
# print(f"AD has finished test evaluation with a test loss of {test_loss}.")

In [ ]:
# Evaluation on the test dataset
# latents = torch.randn((len(test_dl.dataset), model.latent_dim)).to(device)

mu_test = torch.randn(len(test_dl.dataset), latent_dim, device=device, requires_grad=True)
sigma_test = torch.randn(len(test_dl.dataset), latent_dim, device=device, requires_grad=True)
test_latents = torch.nn.parameter.Parameter(torch.stack([mu_test, sigma_test], dim=1)).to(device)

opt = torch.optim.Adam([test_latents], lr=3e-3)
test_loss = evaluate_model(model=model, test_dl=test_dl, opt=opt, latents=test_latents, epochs=300, device=device)
print(f"AD has finished test evaluation with a test loss of {test_loss}.")

In [ ]:
# Randomly sample 5 indices from the test dataset
import random
import utils
random.seed(6)
sampled_indices = random.sample(range(len(test_latents)), 5)

# Extract the corresponding vectors (input data) and their labels
sampled_latents = [test_latents[i] for i in sampled_indices]  # Only selecting input data, not labels

# Convert to a single tensor (optional)
sampled_latents_tensor = torch.stack(sampled_latents)
random_latents_tensor = torch.randn_like(sampled_latents_tensor)

print("Sampled Vectors Shape:", sampled_latents_tensor.shape)  # Should be (5, *) depending on your data shape
print("Random Vectors Shape:", random_latents_tensor.shape)  # Should be (5, *) depending on your data shape

sampled_test_images = model(sampled_latents_tensor).view(-1, 1, 28, 28)
random_test_images = model(random_latents_tensor).view(-1, 1, 28, 28)

print("Sampled Images Shape:", sampled_test_images.shape)  # Should be (5, *) depending on your data shape
utils.save_images(sampled_test_images, "sampled_test_images.png")
utils.save_images(random_test_images, "random_test_images.png")
# import matplotlib.pyplot as plt
# plt.imshow(sampled_test_images.detach().cpu().numpy().reshape(28, 28), cmap='gray')
# plt.show()

In [ ]:
from utils import plot_tsne

result_test_latents = model.reparameterization_trick(test_latents)
utils.plot_tsne(test_ds, result_test_latents, f"tsne_test")
# def plot_tsne(dataset, latents, file_name, plot_title="t-SNE Plot"):
plot_tsne(test_ds, result_test_latents, "just a file variational", plot_title="t-SNE Plot")
